In [ ]:

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os

savefolder = "/Users/your/savefolder"
datafolder = "/Users/your/datafolder"
color = "blue"
exp_name = "zebrafish"

# color = "orange"
# exp_name = "medaka"

data = pd.read_pickle(r"%s/df_%s_0dph_group_lifetime_detail.pkl" % (datafolder, exp_name))
data.tail()

In [ ]:
## visualize turn angle in response to flickering dot motion stimuli

good_fishs_n = 32
stimulus_pattern = []
patterns = [0, 2,4,6,8,10]
# patterns = [1, 3,5,7,9,11]
patterns = [0, 1,2,3,4,5]


trial_num = 20
time_start = 5
time_stop = 25
start_timing = 0
middle_timing = time_start
stop_timing = time_stop
dt = 1./ 90
binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)

good_fishs = []
for fish_n in range(good_fishs_n):
    fish_data = data.loc[(data['fish_index'] == fish_n)]
    if fish_data['center_distance'].mean() < 30:
        good_fishs.append(fish_n)
#         print("this is good fish, %s, %s" % (fishes[fish_n], fish_data['center_distance'].mean()))
selected_fish_data = data.loc[data['fish_index'].isin(good_fishs)]
print(good_fishs)

for pattern in patterns:
    stimulus_pattern.append([pattern])
    

columns = ["bin_n", "density", "pattern", "tbin"]
# dff = pd.DataFrame(columns=columns)\
data_frame = [] #pd.DataFrame(columns=columns)


ave_angles_p, ste_angles_p, ave_angles_p_5_15, ste_angles_p_5_15 = [], [], [], []
# ave_angles_p_5_15, ste_angles_p_5_15 = [], []


for pattern in patterns:
    
    stimulus_pattern_n = pattern
    r_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern)]

    stimulus_pattern_n = pattern + len(patterns)
    l_stim_data = data.loc[(data['stimulus_index'] == pattern + 6)] ## zebrafish
#     l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern + 12)] ## medaka

    df = r_stim_data.copy()
    df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

    merge_data = [l_stim_data, df]
#     merge_data = [df]
    bbig_data = pd.concat(merge_data)
    big_data = bbig_data.loc[(bbig_data['center_distance']<  30)]

    stimuli = []

    for t in binning1:
        if (t - 5) < 0:
            stimuli.append(0)
        elif t < 25:
            stimuli.append(1)
        else:
            stimuli.append(0)
            
    fig, axs = plt.subplots(5, figsize=(4,12))
    
#     axs[0].axvline(x=5, color='black', alpha=0.5, linestyle="--")
#     axs[0].axvline(x=7, color='grey', alpha=0.5, linestyle="--")
    axs[0].plot(binning1, stimuli, color="red")
#     axs[0].set_title("rest: %s, pulse: %s" % (rest, pulse))
    axs[0].set_ylim(-1.1, 1.1)
    axs[0].axhline(y=0, color="black", linestyle="--")
    axs[0].set_ylabel("stimulus")
    axs[0].set_xlim(0.25, 25)

    axs[1].axvline(x=5, color='black', alpha=0.5, linestyle="--")
#     axs[1].axvline(x=25, color='black', alpha=0.5, linestyle="--")
#     axs[1].axvline(x=7, color='grey', alpha=0.5, linestyle="--")
    axs[1].scatter(big_data["start_time_index"], y=big_data["bout_angle"], color=color, alpha=0.3, marker=".", s=0.5, )
#     plt.plot(x, y, color=color, alpha=1,  )
    axs[1].set_ylim(-100, 100)
#     plt.ylim(0, 1)
#     plt.xlim(0, 25)
    axs[1].set_xlim(0.25, 25) #rest + pulse)
    plt.xlim(0.25, 25)
#     plt.axhline(y=0.5, color="black", linestyle="--")
    axs[1].axhline(y=0, color="black", linestyle="--")
    
    plt.tight_layout()
    plt.savefig("%s/fig_%s_lifetime_stim%s_average2.png" % (savefolder, exp_name, pattern), transparent=True)


    plt.show()
    
    fig, axs = plt.subplots(5, figsize=(4,9))
#     axs[0].axvline(x=5, color='black', alpha=0.5, linestyle="--")
#     axs[0].axvline(x=7, color='grey', alpha=0.5, linestyle="--")
    axs[0].plot(binning1, stimuli, color="red")
#     axs[0].set_title("rest: %s, pulse: %s" % (rest, pulse))
    axs[0].set_ylim(-1.1, 1.1)
    axs[0].axhline(y=0, color="black", linestyle="--")
    axs[0].set_ylabel("stimulus")
    axs[0].set_xlim(0.25, 25)

    show_t = []
# show the average turning angle
    # show average of all angles
    ave_angles = []
    std_angles = []
    for t_bin in range(len(analysis_bin)-1):
        data_sub = big_data.loc[(big_data["start_time_index"] > analysis_bin[t_bin]) & (big_data["start_time_index"] < analysis_bin[t_bin+1])]
        show_t.append((analysis_bin[t_bin+1] + analysis_bin[t_bin])/2)
        ave_angles.append(data_sub["bout_angle"].mean())
        std_angles.append(data_sub["bout_angle"].std()/np.sqrt(good_fishs_n))

#     r, q = divmod((25 - 5 - 2), rest + pulse)
#     for r_n in range(int(r)+1):
#         plt.axvspan(5+2+(rest + pulse)*r_n, 5+2+(rest + pulse)*r_n + pulse,
#                     facecolor='red', alpha=0.2, lw=None)

#     axs[2].axvline(x=5, color='black', alpha=0.5, linestyle="--")
#     axs[2].axvline(x=7, color='grey', alpha=0.5, linestyle="--")
    axs[2].axvline(x=5, color='black', alpha=0.5, linestyle="--")
#     axs[2].axvline(x=25, color='black', alpha=0.5, linestyle="--")

    axs[2].fill_between(show_t, np.array(ave_angles) - np.array(std_angles), np.array(ave_angles) + np.array(std_angles), color=color, alpha=0.3)
    axs[2].plot(show_t, ave_angles, color=color, alpha=1, )
#     axs[2].set_ylim(-20, 20)
    axs[2].set_ylim(-10, 40)
    axs[2].axhline(y=0, color="black", linestyle="--")
    axs[2].set_xlim(0.25, 25)
  

    plt.tight_layout()
    plt.savefig("%s/fig_%s_lifetime_stim%s_average2.svg" % (savefolder, exp_name, pattern), transparent=True)

    plt.show()
    
    ## average angle
    data_sub = big_data.loc[(big_data["start_time_index"] > 5) & (big_data["start_time_index"] < 25)]
    ave_angles_p.append(data_sub["bout_angle"].mean()) 
    ste_angles_p.append(data_sub["bout_angle"].std()/np.sqrt(fish_n))
    
    data_subb = big_data.loc[(big_data["start_time_index"] > 5) & (big_data["start_time_index"] < 15)]
    ave_angles_p_5_15.append(data_subb["bout_angle"].mean()) 
    ste_angles_p_5_15.append(data_subb["bout_angle"].std()/np.sqrt(fish_n))
    

In [ ]:
## compare the fitting curve between zebrafish and medaka 
## fitting for every fish, measuring the time constant for every fish, plot in bar plot.
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
from scipy.optimize import curve_fit
from scipy.stats import norm

scale_0 = [0, 0.1, 0.2, 1, 3, 5]
scale =[0.1, 0.2, 1, 3, 5, 20] 
scale_20 =[0, 0.1, 0.2, 1, 3, 5, 20] 
scale_0_fit = np.arange(0,5,0.01)


good_fishs_n =  32
stimulus_pattern = []

patterns_s = [[0, 1,2,3,4,5], [1, 3,5,7,9,11] ]

trial_num = 20
time_start = 5
time_stop = 25
start_timing = 0
middle_timing = time_start
stop_timing = time_stop
dt = 1./ 90
binning1 = np.arange(start_timing, stop_timing-0.1, 0.05)
analysis_bin = np.arange(start_timing, stop_timing-0.1, 0.2)

df_stats = pd.DataFrame(columns=["fish_id", "species", "time_constant"])

species = ["zebrafish", "medaka"]
experiments = ["zebrafish", "medaka"]
colors = ["blue", "orange"]

fig, axs = plt.subplots(1, figsize=(2,1.2))

for experiment_i, experiment in enumerate(experiments):
    data = pd.read_pickle(r"%s/df_%s_0dph_group_lifetime_detail.pkl" % (datafolder, experiment))
    good_fishs_n = 30
    good_fishs = []
    for fish_n in range(good_fishs_n):
        fish_data = data.loc[(data['fish_index'] == fish_n)]
        if fish_data['center_distance'].mean() < 30:
            good_fishs.append(fish_n)
    print(good_fishs)

    # columns2 = ["bin_n", "density", "pattern", "tbin"]
    # dff = pd.DataFrame(columns=columns)\
    # data_frame = [] #pd.DataFrame(columns=columns)


    time_constants = []

    for fish in good_fishs:
        

        ave_angles_p, ave_angles_p_5_15, ave_angles_p_20, ave_angles_p_5_15_20 = [], [], [], []

        patterns = patterns_s[experiment_i]
        
        for pattern in patterns:

            selected_fish_data = data.loc[data['fish_index'] == fish]

            r_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern)]
            if experiment_i == 0:
                l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern + 6)] ## zebrafish
            else:
                l_stim_data = selected_fish_data.loc[(selected_fish_data['stimulus_index'] == pattern + 12)] ## medaka

            df = r_stim_data.copy()
            df['bout_angle'] = r_stim_data['bout_angle'] * (-1)

            merge_data = [l_stim_data, df]
            bbig_data = pd.concat(merge_data)
            big_data = bbig_data.loc[(bbig_data['center_distance']<  30)]

            ## average angle
            data_sub = big_data.loc[(big_data["start_time_index"] > 5) & (big_data["start_time_index"] < 25)]
            ave_angles_p.append(data_sub["bout_angle"].mean()) 
    #         ste_angles_p.append(data_sub["bout_angle"].std()/np.sqrt(fish_n))

            data_subb = big_data.loc[(big_data["start_time_index"] > 5) & (big_data["start_time_index"] < 15)]
            ave_angles_p_5_15.append(data_subb["bout_angle"].mean()) 
    #         ste_angles_p_5_15.append(data_subb["bout_angle"].std()/np.sqrt(fish_n))
    
#         fig, axs = plt.subplots(2, figsize=(3,2))
#         axs[0].plot(scale, ave_angles_p)
#         axs[0].plot(scale, ave_angles_p_5_15)
#         axs[0].set_title("pattern: %s, %s fish: %s" % (pattern, species[experiment_i],fish))

#         ## fitting
        ave_angles_p_fit, ave_angles_p_5_15_fit = [0] + ave_angles_p[:-1], [0] + ave_angles_p_5_15[:-1]
        ave_angles_p_fit_20, ave_angles_p_5_15_fit_20 = [0] + ave_angles_p, [0] + ave_angles_p_5_15

#         axs[1].plot(scale_0, ave_angles_p_fit)
        axs.plot(scale_20, ave_angles_p_5_15_fit_20, color=colors[experiment_i], alpha=0.4)
#         axs.plot(scale_0, ave_angles_p_5_15_fit, color=colors[experiment_i], alpha=0.4)


        def up_exp_cureve(x, *p):
            tau, onset, A, t0 = p
            return A *(1 - np.exp(-(x - t0)/tau)) + onset
axs.set_ylim(-5,45)
axs.set_xlim(-0.1,5)
# plt.savefig("%s/medaka_lifetime_15_25_fit_2_eachfish_2.svg" % (savefolder))

        ## rising
#         p0 = [1, 0, 25, 0.0]
#         coeff_u, var_matrix = curve_fit(up_exp_cureve, scale_20, ave_angles_p_5_15_fit_20, p0=p0, maxfev=5000, bounds = ((0,-1,-15,0),(100,1,30,0.1)))
#         out_fit = up_exp_cureve(scale_0_fit, *coeff_u)
# #         print(coeff_u)
# #         axs[1].plot(scale_0_fit, out_fit, linestyle="--")
# #         plt.tight_layout()
# #     #     plt.savefig("%s/medaka_lifetime_15_25_fit_2.svg" % (savefolder))
# #         plt.show()
#     #     time_constants.append(coeff_u[0])

#         df2 = pd.DataFrame({"fish_id":fish, "species":species[experiment_i], "time_constant":coeff_u[0]},
#                           index=[0])
#         df_stats = pd.concat([df_stats, pd.DataFrame(df2)], ignore_index=True)

# df_stats

In [ ]:
## dot plot of time constants of lifetime

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

tau_name = "time_constants"

list_z_df = df_stats.loc[df_stats["species"]=="zebrafish"]
list_z = list_z_df['time_constant'].tolist()
list_m_df = df_stats.loc[df_stats["species"]=="medaka"]
list_m = list_m_df['time_constant'].tolist()

# Combine the data into a single DataFrame
data = {
    'Value': list_z + list_m,
    'Group': ['z'] * len(list_z) + ['m'] * len(list_m)
}

colors = ["blue", "orange"]
# Create the swarm plot
plt.figure(figsize=(1.6, 2))
sns.swarmplot(x='Group', y='Value', data=data, alpha=0.8, palette=colors)

# Add the average value lines
avg_list1 = np.mean(list_z)
avg_list2 = np.mean(list_m)

print(avg_list1, avg_list2, np.std(list_z), np.std(list_m))

# Add short average value lines
line_length = 0.3  # This defines the length of the lines

# Plot short horizontal lines for averages
plt.plot([0 - line_length, 0 + line_length], [avg_list1, avg_list1], color='blue', linestyle='-', linewidth=2,)# label=f'Avg List 1: {avg_list1:.2f}')
plt.plot([1 - line_length, 1 + line_length], [avg_list2, avg_list2], color='darkorange', linestyle='-', linewidth=2,)# label=f'Avg List 2: {avg_list2:.2f}')

# plt.ylim(0.1, 1.2)
plt.tight_layout()
# plt.savefig("%s/uniDirection/fig_%s_fit_tau_2.svg" % (savefolder, tau_name),
#                         transparent=True)
plt.show()
